In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('Exercise_1_cont').getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

In [ ]:
file_path = '/content/restaurant-orders.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [ ]:
df.select(count('*').alias("Total_rows")).show()

+----------+
|Total_rows|
+----------+
|    119183|
+----------+



In [ ]:
df.select(count('*').alias("Total_rows"), countDistinct("Order ID").alias("Total_orders")).show()

+----------+------------+
|Total_rows|Total_orders|
+----------+------------+
|    119183|       19658|
+----------+------------+



In [ ]:
df.select(count('*').alias("Total_rows"),
          countDistinct("Order ID").alias("Total_orders"), sum("Quantity"), round(avg("Product Price"),2)).show()



+----------+------------+-------------+----------------------------+
|Total_rows|Total_orders|sum(Quantity)|round(avg(Product Price), 2)|
+----------+------------+-------------+----------------------------+
|    119183|       19658|       148844|                        5.11|
+----------+------------+-------------+----------------------------+



In [ ]:
df.selectExpr("count(*) as Total_rows",
              "count(distinct 'Order ID') as Total_orders",
              "sum(Quantity)",
              "round(avg('Product Price'),2)").show()

+----------+------------+-------------+----------------------------+
|Total_rows|Total_orders|sum(Quantity)|round(avg(Product Price), 2)|
+----------+------------+-------------+----------------------------+
|    119183|           1|       148844|                        NULL|
+----------+------------+-------------+----------------------------+



In [ ]:
df.createOrReplaceTempView("orders")

In [ ]:
spark.sql("""
    SELECT
        count(*) as Total_rows,
        count(distinct `Order ID`) as Tatal_orders,
        sum(Quantity),
        round(avg(`Product Price`),2)
    FROM orders
""").show()

+----------+------------+-------------+----------------------------+
|Total_rows|Tatal_orders|sum(Quantity)|round(avg(Product Price), 2)|
+----------+------------+-------------+----------------------------+
|    119183|       19658|       148844|                        5.11|
+----------+------------+-------------+----------------------------+



In [ ]:
df.groupBy("Order ID").agg(sum("Quantity").alias("Total_quantity")).collect()  #show() --> 20 items

[Row(Order ID=25517, Total_quantity=4),
 Row(Order ID=24663, Total_quantity=6),
 Row(Order ID=24347, Total_quantity=13),
 Row(Order ID=24171, Total_quantity=4),
 Row(Order ID=21700, Total_quantity=10),
 Row(Order ID=21220, Total_quantity=6),
 Row(Order ID=20924, Total_quantity=6),
 Row(Order ID=20497, Total_quantity=15),
 Row(Order ID=20382, Total_quantity=9),
 Row(Order ID=19553, Total_quantity=8),
 Row(Order ID=19204, Total_quantity=8),
 Row(Order ID=18944, Total_quantity=9),
 Row(Order ID=18498, Total_quantity=7),
 Row(Order ID=12046, Total_quantity=12),
 Row(Order ID=11858, Total_quantity=12),
 Row(Order ID=11033, Total_quantity=5),
 Row(Order ID=10623, Total_quantity=10),
 Row(Order ID=10362, Total_quantity=7),
 Row(Order ID=10206, Total_quantity=14),
 Row(Order ID=9900, Total_quantity=18),
 Row(Order ID=7993, Total_quantity=11),
 Row(Order ID=7880, Total_quantity=9),
 Row(Order ID=7554, Total_quantity=7),
 Row(Order ID=7253, Total_quantity=17),
 Row(Order ID=6654, Total_quantity=

In [ ]:
df.groupBy("Order ID").agg(sum("Quantity").alias("Total_quantity")).filter(col("Order ID") == 25583).show()

+--------+--------------+
|Order ID|Total_quantity|
+--------+--------------+
|   25583|            14|
+--------+--------------+



In [ ]:
df.groupBy("Order ID").agg(sum("Quantity").alias("Total_quantity"), round(sum(df.Quantity * col("Product Price")),2).alias("Total_amount")).show()

+--------+--------------+------------+
|Order ID|Total_quantity|Total_amount|
+--------+--------------+------------+
|   25517|             4|       24.35|
|   24663|             6|       24.65|
|   24347|            13|        30.5|
|   24171|             4|        17.9|
|   21700|            10|       46.15|
|   21220|             6|       19.65|
|   20924|             6|        25.8|
|   20497|            15|       52.75|
|   20382|             9|        20.9|
|   19553|             8|       25.85|
|   19204|             8|        28.7|
|   18944|             9|        25.9|
|   18498|             7|       27.75|
|   12046|            12|       35.05|
|   11858|            12|        32.0|
|   11033|             5|        20.3|
|   10623|            10|       17.25|
|   10362|             7|        27.9|
|   10206|            14|        54.8|
|    9900|            18|       72.15|
+--------+--------------+------------+
only showing top 20 rows



In [ ]:
df.groupBy("Order ID").agg(expr("sum(Quantity) as Total_quantity"), expr("round(sum(Quantity * `Product Price`),2) as Total_amount")).show()

+--------+--------------+------------+
|Order ID|Total_quantity|Total_amount|
+--------+--------------+------------+
|   25517|             4|       24.35|
|   24663|             6|       24.65|
|   24347|            13|        30.5|
|   24171|             4|        17.9|
|   21700|            10|       46.15|
|   21220|             6|       19.65|
|   20924|             6|        25.8|
|   20497|            15|       52.75|
|   20382|             9|        20.9|
|   19553|             8|       25.85|
|   19204|             8|        28.7|
|   18944|             9|        25.9|
|   18498|             7|       27.75|
|   12046|            12|       35.05|
|   11858|            12|        32.0|
|   11033|             5|        20.3|
|   10623|            10|       17.25|
|   10362|             7|        27.9|
|   10206|            14|        54.8|
|    9900|            18|       72.15|
+--------+--------------+------------+
only showing top 20 rows



In [ ]:
spark.sql("""
    SELECT
        `Order ID`,
        sum(Quantity) as Total_quantity,
        round(sum(Quantity * `Product Price`),2) as Total_amount
    FROM orders
    GROUP BY `Order ID`
""").show()

+--------+--------------+------------+
|Order ID|Total_quantity|Total_amount|
+--------+--------------+------------+
|   25517|             4|       24.35|
|   24663|             6|       24.65|
|   24347|            13|        30.5|
|   24171|             4|        17.9|
|   21700|            10|       46.15|
|   21220|             6|       19.65|
|   20924|             6|        25.8|
|   20497|            15|       52.75|
|   20382|             9|        20.9|
|   19553|             8|       25.85|
|   19204|             8|        28.7|
|   18944|             9|        25.9|
|   18498|             7|       27.75|
|   12046|            12|       35.05|
|   11858|            12|        32.0|
|   11033|             5|        20.3|
|   10623|            10|       17.25|
|   10362|             7|        27.9|
|   10206|            14|        54.8|
|    9900|            18|       72.15|
+--------+--------------+------------+
only showing top 20 rows



In [ ]:
# thống kê số lượng sản phẩm bán ra theo từng loại xét theo năm

In [ ]:
# 1. Convert 'Order Date' to date type
# Assuming 'Order Date' is in 'MM/dd/yyyy HH:mm' format, adjust if necessary
# The original format "MM/dd/yyyy" was incorrect and caused the error.
df = df.withColumn("Order Date", to_date(col("Order Date"), "MM/dd/yyyy HH:mm"))

# 2. Extract the year from 'Order Date'
df = df.withColumn("Order Year", year(col("Order Date")))

# 3. Group by 'Item Name' and 'Order Year' and sum the quantities
product_sales_by_year = df.groupBy("Item Name", "Order Year").agg(
    sum("Quantity").alias("Total Quantity")
)

# 4. Display the results
product_sales_by_year.show()

+--------------------+----------+--------------+
|           Item Name|Order Year|Total Quantity|
+--------------------+----------+--------------+
|          Keema Naan|      2019|           217|
|        Graps Specil|      2017|             9|
|          Mint Sauce|      NULL|          1985|
|      Dupiaza - Lamb|      NULL|            45|
|    Vegetable Dansak|      2018|            12|
|       Jeera Chicken|      2019|             6|
|               Korma|      2017|           438|
|Bhuna - chicken-t...|      NULL|             8|
|         Prawn Puree|      2017|            80|
|     Garlic Mushroom|      2019|             7|
|        Royal Paneer|      2018|            31|
|     Vegetable Samba|      2018|             1|
|     Garlic Mushroom|      2017|             7|
|    Pathia - chicken|      2018|             7|
|     Bhuna - Chicken|      NULL|           149|
|     Mushroom Bhajee|      NULL|           370|
|         Chana Chaat|      2019|             7|
|        Aloo Dupiaz

---

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
sprk = SparkSession.builder.appName('Exercise_1_cont').getOrCreate()

In [29]:
file_path = '/content/sample_data/restaurant-orders.csv'
df = sprk.read.csv(file_path, header=True, inferSchema=True)

In [30]:
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Item Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Product Price: double (nullable = true)
 |-- Total products: integer (nullable = true)



In [19]:
transformed_df = df.withColumn("Order Date (new)", to_date(col("Order Date"), "MM/dd/yyyy HH:mm"))
transformed_df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Item Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Product Price: double (nullable = true)
 |-- Total products: integer (nullable = true)
 |-- Order Date (new): date (nullable = true)



In [20]:
transformed_df.show()

+--------+----------------+--------------------+--------+-------------+--------------+----------------+
|Order ID|      Order Date|           Item Name|Quantity|Product Price|Total products|Order Date (new)|
+--------+----------------+--------------------+--------+-------------+--------------+----------------+
|   25583|03/08/2019 21:58|Tandoori Mixed Grill|       1|        11.95|            12|      2019-03-08|
|   25583|03/08/2019 21:58|        Madras Sauce|       1|         3.95|            12|      2019-03-08|
|   25583|03/08/2019 21:58|       Mushroom Rice|       2|         3.95|            12|      2019-03-08|
|   25583|03/08/2019 21:58|         Garlic Naan|       1|         2.95|            12|      2019-03-08|
|   25583|03/08/2019 21:58|             Paratha|       1|         2.95|            12|      2019-03-08|
|   25583|03/08/2019 21:58|          Plain Rice|       1|         2.95|            12|      2019-03-08|
|   25583|03/08/2019 21:58|         Prawn Puree|       1|       

In [11]:
df = df.withColumn("Order Date", col("Order Date").cast("timestamp"))
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Order Date: timestamp (nullable = true)
 |-- Item Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Product Price: double (nullable = true)
 |-- Total products: integer (nullable = true)



In [12]:
df.show()

+--------+----------+--------------------+--------+-------------+--------------+
|Order ID|Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+----------+--------------------+--------+-------------+--------------+
|   25583|      NULL|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|      NULL|        Madras Sauce|       1|         3.95|            12|
|   25583|      NULL|       Mushroom Rice|       2|         3.95|            12|
|   25583|      NULL|         Garlic Naan|       1|         2.95|            12|
|   25583|      NULL|             Paratha|       1|         2.95|            12|
|   25583|      NULL|          Plain Rice|       1|         2.95|            12|
|   25583|      NULL|         Prawn Puree|       1|         4.95|            12|
|   25583|      NULL|       Plain Papadum|       1|          0.8|            12|
|   25583|      NULL|       Mango Chutney|       2|          0.5|            12|
|   25583|      NULL|       

In [31]:
df = df.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Order Date: timestamp (nullable = true)
 |-- Item Name: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Product Price: double (nullable = true)
 |-- Total products: integer (nullable = true)



In [32]:
df.show(10)

+--------+-------------------+--------------------+--------+-------------+--------------+
|Order ID|         Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+-------------------+--------------------+--------+-------------+--------------+
|   25583|2019-03-08 21:58:00|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|2019-03-08 21:58:00|        Madras Sauce|       1|         3.95|            12|
|   25583|2019-03-08 21:58:00|       Mushroom Rice|       2|         3.95|            12|
|   25583|2019-03-08 21:58:00|         Garlic Naan|       1|         2.95|            12|
|   25583|2019-03-08 21:58:00|             Paratha|       1|         2.95|            12|
|   25583|2019-03-08 21:58:00|          Plain Rice|       1|         2.95|            12|
|   25583|2019-03-08 21:58:00|         Prawn Puree|       1|         4.95|            12|
|   25583|2019-03-08 21:58:00|       Plain Papadum|       1|          0.8|            12|
|   25583|

In [ ]:
transformed_df.createOrReplaceGlobalTempView("orders")

In [33]:
df.createOrReplaceTempView("orders")

In [22]:
sprk.sql("SELECT * FROM orders").show

<bound method DataFrame.show of DataFrame[Order ID: int, Order Date: string, Item Name: string, Quantity: int, Product Price: double, Total products: int]>

In [ ]:
sprk.sql("SELECT `Item Name`, Quantity, date_format(`Order Date`, 'yyyy') as Year FROM orders").show()

+---------+--------+----+
|Item Name|Quantity|Year|
+---------+--------+----+
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       2|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       2|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
|Item Name|       1|2019|
+---------+--------+----+
only showing top 20 rows



In [25]:
sprk.sql("SELECT `Item Name`, date_format(`Order Date`, 'yyyy') as Year, sum(Quantity) as Amount FROM orders GROUP BY `Item Name`, Year").show()

+--------------------+----+------+
|           Item Name|Year|Amount|
+--------------------+----+------+
|          Mint Sauce|NULL|  3208|
|      Dupiaza - Lamb|NULL|    68|
|    Lamb Tikka Balti|NULL|    95|
|   Vegetable Biryani|NULL|   234|
|Bhuna - chicken-t...|NULL|    10|
|     Mushroom Bhajee|NULL|   584|
|     Bhuna - Chicken|NULL|   239|
|           Saag Aloo|NULL|  2089|
|Malaya - Chicken ...|NULL|    16|
|             Dhansak|NULL|   512|
|        Madras Sauce|NULL|   233|
|            Vindaloo|NULL|   387|
|         Prawn Puree|NULL|   844|
|      Grapes Special|NULL|   368|
|             Dupiaza|NULL|   261|
|     Chicken Biryani|NULL|  1071|
|   Lamb Tikka (Main)|NULL|    88|
|       Mushroom Rice|NULL|  3424|
|Cylon - chicken-t...|NULL|     1|
|Korma - Chicken T...|NULL|    80|
+--------------------+----+------+
only showing top 20 rows



In [38]:
sprk.sql("""
      SELECT `Item Name`, date_format(`Order Date`, 'yyyy') as Year, sum(Quantity) as Amount
      FROM orders GROUP BY `Item Name`, Year ORDER BY Year""").groupBy("Item Name").pivot("Year").sum("Amount").show()

+--------------------+----+----+----+----+----+
|           Item Name|null|2016|2017|2018|2019|
+--------------------+----+----+----+----+----+
|      Dupiaza - Lamb|  45|NULL|NULL|   8|  15|
|          Mint Sauce|1985| 141| 392| 404| 286|
|    Lamb Tikka Balti|  49|   2|   9|  24|  11|
|Bhuna - chicken-t...|   8|NULL|NULL|   2|NULL|
|   Vegetable Biryani| 143|  16|  18|  26|  31|
|     Bhuna - Chicken| 149|NULL|NULL|  42|  48|
|     Mushroom Bhajee| 370|  22|  67|  73|  52|
|           Saag Aloo|1249| 116| 265| 284| 175|
|Malaya - Chicken ...|  13|NULL|NULL|   1|   2|
|             Dhansak| 308|  50| 114|  40|NULL|
|       Korma - prawn|   2|NULL|NULL|   1|NULL|
|        Madras Sauce| 135|   3|  15|  38|  42|
|            Vindaloo| 230|  35|  93|  29|NULL|
|         Prawn Puree| 522|  37|  80| 116|  89|
|   Saag - king-prawn|NULL|NULL|NULL|   1|NULL|
|      Grapes Special| 218|  17|  42|  56|  35|
|  Korma - King Prawn|  17|NULL|NULL|   2|   5|
|        Saag - Prawn|NULL|NULL|NULL|   